<a href="https://colab.research.google.com/github/SaeidDaliriSusefi/SoilMoisture-Landsat8/blob/main/Main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import ee
import geemap
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
ee.Authenticate()
ee.Initialize(project="ee-saeiddalirisu", opt_url='https://earthengine-highvolume.googleapis.com')

In [ ]:
# Define variables
country_name = 'Italy'
region_name = 'Lombardia'
city_name = 'Lecco'
time_start = '2024-01-01'
time_end = '2024-12-30'

In [ ]:
# Load and filter administrative boundaries
roi = (ee.FeatureCollection("FAO/GAUL_SIMPLIFIED_500m/2015/level2")
          .filter(ee.Filter.eq('ADM0_NAME', country_name))
          .filter(ee.Filter.eq('ADM1_NAME', region_name))
          .filter(ee.Filter.eq('ADM2_NAME', city_name)))


In [ ]:
landsat = (ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")
    .filterBounds(roi)
    .filterDate(time_start, time_end)
    .filter(ee.Filter.eq('WRS_PATH', 194))
    .filter(ee.Filter.eq('WRS_ROW', 28))
    .filter(ee.Filter.lt('CLOUD_COVER', 10))
    .map(lambda img: (
    img.select('SR_B.*').multiply(ee.Number(img.get('REFLECTANCE_MULT_BAND_3')))
    .add(ee.Number(img.get('REFLECTANCE_ADD_BAND_3')))
    .normalizedDifference(['SR_B5', 'SR_B4']).rename('ndvi')  # Calculate NDVI
    .addBands(
        img.select('ST_B10').multiply(0.00341802).add(149).rename('lst')  # Calculate LST
    )
    .copyProperties(img, img.propertyNames())  # Copy image properties
))
)


In [ ]:
# Apply mask based on NDVI condition and return updated LST
lst_full_cover = landsat.map(lambda img: (
    img.select('lst').updateMask(img.select('ndvi').gt(0.3))  # Apply mask where NDVI > 0.3
    .rename('lst')  # Rename the LST band
    .copyProperties(img, img.propertyNames())  # Copy properties from the original image
))

# The lst_full_cover will now contain the images with the masked LST based on NDVI > 0.3
lst_full_cover

In [ ]:
# Apply mask based on NDVI condition and return updated LST
lst_bareland = landsat.map(lambda img: (
    img.select('lst').updateMask(img.select('ndvi').gte(0).And(img.select('ndvi').lt(0.2)))  # Apply mask where NDVI is between 0 and 0.2
    .rename('lst')  # Rename the LST band
    .copyProperties(img, img.propertyNames())  # Copy properties from the original image
))

# The lst_bareland will now contain the images with the masked LST based on NDVI between 0 and 0.2
lst_bareland


In [ ]:
id = ee.Number(lst_bareland.max().reduceRegion(
    reducer=ee.Reducer.max(),
    geometry=roi,
    scale=100
).get('lst'))

vd = ee.Number(lst_full_cover.max().reduceRegion(
    reducer=ee.Reducer.max(),
    geometry=roi,
    scale=100
).get('lst'))

iw = ee.Number(lst_bareland.min().reduceRegion(
    reducer=ee.Reducer.min(),
    geometry=roi,
    scale=100
).get('lst'))

vw = ee.Number(lst_full_cover.min().reduceRegion(
    reducer=ee.Reducer.min(),
    geometry=roi,
    scale=100
).get('lst'))


sd = id.subtract(vd)
sw = iw.subtract(vw)

In [ ]:
sm = landsat.map(lambda img: (
    img.expression(
        '(id + sd * ndvi - lst) / (id - iw + (sd - sw) * ndvi)', {
            'id': id,
            'sd': sd,
            'ndvi': img.select('ndvi'),
            'lst': img.select('lst'),
            'iw': iw,
            'sw': sw
        }).rename('soil_moisture')
    .copyProperties(img, img.propertyNames())
))


In [ ]:

def get_band_values(img):
    image_date = img.get('system:time_start')
    image_date = ee.Date(image_date).format('YYYY-MM-dd').getInfo()

    region_values = img.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=roi,
        scale=30,
        maxPixels=1e8
    )

    return (image_date, region_values)

band_values_dict = {}

for i in range(sm.size().getInfo()):
    img = sm.toList(sm.size()).get(i)
    date, band_values = get_band_values(ee.Image(img))
    band_values_dict[date] = band_values.getInfo()  # Add the data to the dictionary

import matplotlib.pyplot as plt


dates = []
soil_moisture_values = []

for date, band_values in band_values_dict.items():
    if 'soil_moisture' in band_values:  # Check if the soil moisture band exists
        soil_moisture_values.append(band_values['soil_moisture'])
        dates.append(date)

date_nums = np.arange(len(dates))

slope, intercept = np.polyfit(date_nums, soil_moisture_values, 1)
trendline = np.polyval([slope, intercept], date_nums)

plt.figure(figsize=(20, 6))
plt.plot(dates, soil_moisture_values, marker='o', linestyle='-', color='b', label='Soil Moisture')
plt.plot(dates, trendline, color='r', linestyle='--', label='Trendline')


plt.xlabel('Date')
plt.ylabel('Soil Moisture')
plt.title('Soil Moisture over Time')
plt.xticks(rotation=45)
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.savefig('soil_moisture_trendline.png', format='png')